# Chapter 1 Foundation of Probability Theory Part III

#### *Zhuo Jianchao* 

Feb 6, 2020 *Rev 1*

You can pass this Part if you're very comfortable with the concept of **Permutation** and **Combination**.

## Fundamental Theorem of Counting

If a random experiment should be done in *k* seperate steps, and the $i^{th}$ step yields $n_i$ outcomes, then the total possible outcomes of the random experiment is $
n_{1} \times n_{2} \times \cdots \times n_{k}=\Pi_{i=1}^{k} n_{i}$.

This theorem states if we want to find the number of basic outcomes of a seperate multi-step random experiment, take product of the number of possible outcomes for each step.

## Methods of Counting

Why counting matters? Until now, the probability function in our example of random experiment can be attained by counting the number of basic outcomes in an given event and dividing it by the number of total basic outcomes in the sample space. If you're still confused by such statement, recalling the Example 6.1 in Part II which we will discuss a little bit deeper below. If going fine, skip the example.

### Example 6.1 in Part II

Roll a coin, record which face showing up. It must be a head $H$ or a tail $T$, then the sample space is $\{H,T\}$, and the sigma algebra associated with it is $\{\emptyset,H,T,\{H,T\}\}$. 

Recall that every element in the sigma algebra is an event.
* $\{H\}$ in the sigma algebra means rolling a head;
* $\{T\}$ stands for rolling a tail;
* $\{H,T\}$ represents rolling a head **or** a tail, which is certain because either one of a head or a tail must occur;
* $\{\emptyset\}$ means neither a head nor a tail occurs, which is impossible.

There are two basic outcomes in the sample space with even, or the same likelihood of occurrence, then event $\{H\}$ is one outcome in the total two possible outcomes, hence has probbility of $\frac{1}{2}$. The same to two extreme cases where $\{H,T\}$ covers all outcomes hence has probability of $1$ and $\{\emptyset\}$ contains naught outcomes hence 0.

### Method 0 Ordering

If there are three card on a desk marked $A, B, C$ respectively. You randomly choose one of them as the first card, then choose another one as the second card, then the third one. Once you done it, you get a sequence of cards, with specific *order*.

In [1]:
"""
randomly select one from [A,B,C]
"""
function select_one_card()
    card = ['A','B','C']
    rand(card,1)
end


"""
do it three times to get a sequence of cards
"""
function select_cards()
    order = []
    card₁ = ['A','B','C']
    first = rand(card₁,1)[1]
    card₂ = filter!(i->i≠first, card₁)
    second = rand(card₂,1)[1]
    card₃ = filter!(i->i≠second, card₂) 
    third = rand(card₃,1)[1]
    push!(order, first, second, third)
end

select_cards

Calling the function `select_cards` yields a specific ordering of our cards.

In [2]:
select_cards()

3-element Array{Any,1}:
 'B'
 'A'
 'C'

But there are how many distinct orderings of cards? For the first position, there are three possible cards; now there are two cards remaining hence two possible cards for the second position; and the last one fits the third position. Use the fundamental theorem of counting for generating a sequence of three cards in three steps, there are $3 \times 2 \times 1=6$ possible orderings in total. Use simulation to check our finding.

In [3]:
"""
repeat the random experiment n times, and return number of distinct orderings of sequence of cards
"""
function check_order(n)
    count = Dict()
    for i ∈ 1:n
        order = select_cards()
        if !(order in values(count))
            count[i] = order
        end
    end
    return length(count)
end

check_order

***Tips***: The reason why we use a *dictionary* instead of an *array* as the container of possible orderings is that each time we generate a sequence, we need to search in the container whether the sequence is already in it. Searching in an array is to check elements in the array one by one, until the target is identified or the end is reached. As the array becomes longer, containing more elements, it takes longer time to finish searching. For a dictionary, it is constructed in a different way, making the time for searching relatively the same for a larger dictionary. 

Even though, in this case, we only consider the ordering of three items, it's still a good habit to use a more efficient method as it can be easily extended for a larger example.

We simulate repeating a thousand times, by calling ```check_order(1000)```, which yields 6, our theoretical number of possible orderings.

In [4]:
check_order(1000)

6

The example can be extended to be the number of ordering of *n* cards instead of only 3. In the extended example, for the first position, there are $n$ possible outcomes; for the second one, there are $(n-1)$; for the $3^{rd}$ position, there are $(n-2)$, all the way down to the $n^{th}$ position, there are $(n-n+1)$ which is $1$. By the fundamental theorem of counting, all the positions combined, there are 
$$\underbrace{n \times (n-1) \times (n-2) \times \dots \times 2 \times 1}_n$$

also denoted as $n!$ possibilities. The $n!$ is called a **factorial** of n. And we state that factorial of zero is 1, that is

$$0!=1$$

### Method 1 Permutation

Similar to previous example, this time let's say, we want to select **two** cards from total **four** different cards then how many distinct **ordered** sequence there will be?

Because order matters, for the first position there are four possible outcomes; for the second, there are three; then the total possiblities is $4 \times 3 = 12$.

For a more abstract example, how many ordered sequences if we select $m$ cards from total $n$ different cards?

As the same way, for the first position, there are $n$ possible outcomes, for the second, there are $n-1$, all the way down to the $m-1^{th}$ position, there are $n-(m-1)+1=n-m+2$ and for the last $m^{th}$ position, there are $n-m+1$ possibilities.

Combine all the *n* positions, we have total possibilities of 
$$\underbrace{n \times (n-1) \times \dots \times (n-m+2) \times (n-m+1)}_m$$

It's a quite general case because we use $P_n^{m}$ to denote the number of **permutations**, that is, the ordered sequence with *m* items selected from *n* total items, then

$$P_n^{m}=\underbrace{n \times (n-1) \times \dots \times (n-m+2) \times (n-m+1)}_m$$

We can incorporate the formula with the factorial so that it can be expressed in a more compact way:
$$P_n^{m}=\frac{n!}{(n-m)!}$$

Let's generate a function that calculate it.

In [5]:
"""
takes two arguments, the larger being total number of items and the smaller being number of items of selection.
"""
function permutation(a::Int64,b::Int64)
    # check for validation of arguments
    if a * b < 0
        error("invalid n or m.")
    end
    # assign values
    if a ≥ b
        n, m = a, b
    else
        n, m = b, a
    end
    product = 1
    for i in 0:m-1
        product = product * (n-i)
    end
    return product
end


"""
another function to calculate number of permutations but incorporated in factorial.
"""
function permutation_fac(a::Int64,b::Int64)
    if a * b < 0
        error("invalid n or m.")
    end
    if a ≥ b
        n, m = a, b
    else
        n, m = b, a
    end
    return factorial(n)/factorial(n-m)
end

permutation_fac

In [6]:
permutation(2,4) == permutation_fac(2,4)

true

Let's use `permutation` to check answer in the example.

In [7]:
permutation(2,4) == 12 && print("🎉bingo~")

🎉bingo~

### Method 2 Combinations

In permutation, order counts. If two sequences are deemed identical if their elements are the same no matter what the order of elements is, the sequence is called a **combination**. 

To select three letters from $A,B,C,D$, the permutations are $ABC,ABD,ACD,BCD$, and other five sets of permutations with each item be in a different order, that is $BCA,BDA,CDA,CDB$, and $ACB,ADB,ADC,BDC$ and $CBA,DBA,DCA,DCB$, etc., totally 24 permutations in 6 different orders.

If we care about the number of combination, in which case $ABC$ is exactly the same as $BCA$ and $ACB$, the number of distinct sequence "trisected" to be only 4.

From this example we can know that *the number of combinations* is *the number of permutation* divided by something.

That something in this example, is the number of ordering of our three-time selection, take $ABC$ for example. Because we are to select a sequence of three letters like $ABC$, the order of which can be in $3!$ ways, that is 6. So the there are 4 combinations of sequence without order and 6 orders totalling $4 \times 6 = 24$ permutations.

Conclusion, the number of combination from a selection of *m* units from total *n* units is the *number of permutations* of *m* units out of *n* units, that is $P_n^m$, divided by the number of orders our selection of *m* units out of *m* units is, that is $P_m^m$.

Use $C_n^m$ to denote the number of combinations of selecting *m* units from *n* units, then
$$C_n^m=\frac{P_n^m}{P_m^m}=\frac{\frac{n!}{(m-n)!}}{m!}=\frac{n!}{m!(n-m)!}$$

We use $C_n^m$ and $\binom{n}{m}$ to denote the number of combinations of choosing *m* items from *n* items interchangeably, but be aware the position of each parameter.

In [8]:
"""
takes two arguments, the larger being total number of items and the smaller being number of items of selection.
"""
function combination(a::Int64,b::Int64)
    # check for validation of arguments
    if a * b < 0
        error("invalid n or m.")
    end
    # assign values
    if a ≥ b
        n, m = a, b
    else
        n, m = b, a
    end
    return permutation(a,b) ÷ factorial(m)::Int64
end

combination

In [9]:
combination(4,3) == 4 && print("👍 well done!")

👍 well done!